# Titanic Competition
This jupyter notebook includes the training of ML model used to predict the survival of passengers aboard the titanic

In [46]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [47]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [48]:
def extract_features(df):
    features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]
    X = pd.get_dummies(df[features])
    X["Fare"] = df["Fare"]
    return X

In [49]:
X = extract_features(train)
y = train["Survived"]

In [50]:
rfc = RandomForestClassifier()

In [51]:
forest_params = [{
    "criterion": ["entropy", "gini", "log_loss"],
    "max_leaf_nodes": [5, 50, 500, 500],
    "n_estimators": [10, 100, 1000],
    "max_features": ["sqrt", "log2", None]
}]

In [52]:
clf = GridSearchCV(rfc, forest_params, cv=10, scoring="accuracy")

In [53]:
clf.fit(X, y)
print(clf.best_params_)

{'criterion': 'gini', 'max_features': 'log2', 'max_leaf_nodes': 50, 'n_estimators': 100}


In [54]:
model = clf.best_estimator_
model.fit(X, y)

RandomForestClassifier(max_features='log2', max_leaf_nodes=50)

In [55]:
def verify(model, X, y):
    verification = model.predict(X)
    error = np.sum(abs(verification - y)) / len(y)
    print(
        f"Percentage Error: {error:.0%}"
    )

In [56]:
verify(model, X, y)

Percentage Error: 12%


In [80]:
X_test = extract_features(test)

In [81]:
X_test.isnull().sum()

Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Fare          1
dtype: int64

In [98]:
# Replace the missing value with the median
# This is bad if the specific entry is extraordinary
X_test.loc[X_test["Fare"].isnull(), "Fare"] = np.median(X_test["Fare"].dropna())

In [99]:
predictions = model.predict(X_test)

In [100]:
output = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})
output.to_csv('submission_iter3.csv', index=False)